In [1]:
import aocd
from aocd.models import Puzzle
from aocd import submit
from collections import defaultdict
import itertools

current_day = 23
current_year = 2022
puzzle = Puzzle(year=current_year, day=current_day)

In [2]:
test_input = '''....#..
..###.#
#...#.#
.#...##
#.###..
##.#.##
.#..#..'''

# test_input = '''.....
# ..##.
# ..#..
# .....
# ..##.
# .....'''

def get_input_data(test = 0):
    if test:
        input_data = test_input
    else:
        input_data = puzzle.input_data
    return input_data

In [3]:
def crange(start, num_steps, modulo):
    '''
    circular range - starting at 'start', going for num_steps iterations, and output is modulo w.r.t. 'modulo'
    '''
    index = start % modulo
    for i in range(num_steps):
        yield index
        index = (index + 1) % modulo

In [4]:
class Ground:

    def __init__(self, ground_description):
        ground_description = ground_description.splitlines()
        
        elf_id = 0
        self.elf_locations, self.new_elf_locations = {}, {}
        for row, line in enumerate(ground_description):
            for col, char in enumerate(line):
                if char == '#':
                    self.elf_locations[(row,col)] = elf_id
                    elf_id += 1
        self.num_elves = elf_id

        self.rules = [self.north_neighbors, self.south_neighbors, self.west_neighbors, self.east_neighbors]
        # directions = ['north','south','west','east']
        self.directions = [(-1,0), (+1,0), (0,-1), (0,+1)]
        self.num_rules = len(self.rules)
        self.rule_start_idx = -1
        self.elf_proposals = defaultdict(list)
        self.num_rounds_completed = 0
        
    def west_neighbors(self, row, col):
        yield from [(row-1,col-1),(row,col-1),(row+1,col-1)]

    def east_neighbors(self, row, col):
        yield from [(row-1,col+1),(row,col+1),(row+1,col+1)]

    def south_neighbors(self, row, col):
        yield from [(row+1,col-1),(row+1,col),(row+1,col+1)]

    def north_neighbors(self, row, col):
        yield from [(row-1,col-1),(row-1,col),(row-1,col+1)]

    def neighborhood(self, row, col):
        for drow in [-1,0,1]:
            for dcol in [-1,0,1]:
                if drow==0 and dcol==0:
                    continue
                else:
                    yield (row+drow,col+dcol)
        

    def __repr__(self):
        max_row = max(key[0] 
                      for key in self.elf_locations)
        max_col = max(key[1] 
                      for key in self.elf_locations)
        min_row = min(key[0] 
                      for key in self.elf_locations)
        min_col = min(key[1] 
                      for key in self.elf_locations)

        strings = []
        for row in range(min_row, max_row+1):
            strings.append(''.join('#' 
                                  if (row,col) in self.elf_locations 
                                  else '.'
                                  for col in range(min_col, max_col+1)))
        return '\n'.join(strings)

    def draw(self):
        print(self)


    def count_empty_tiles(self):
        max_row = max(key[0] 
                      for key in self.elf_locations)
        max_col = max(key[1] 
                      for key in self.elf_locations)
        min_row = min(key[0] 
                      for key in self.elf_locations)
        min_col = min(key[1] 
                      for key in self.elf_locations)
        # return sum((row,col) not in locations 
        #            for row in range(min_row, max_row+1) 
        #            for col in range(min_col, max_col+1))
        return (max_row-min_row+1) * (max_col-min_col+1) - len(self.elf_locations)
            
            
    def process_round(self):
        # first half of the round - make proposals
        self.rule_start_idx = (self.rule_start_idx + 1) % self.num_rules
        num_need_to_move = 0
        self.elf_proposals.clear()
        for row,col in self.elf_locations:
            if any(neighbor in self.elf_locations for neighbor in self.neighborhood(row,col)):
                num_need_to_move += 1
                # all neighbors are not empty - have to propose a move
                for rule_id in crange(self.rule_start_idx, 
                                      num_steps=self.num_rules, 
                                      modulo=self.num_rules):
                    nbr_func = self.rules[rule_id]
                    if all(nbr not in self.elf_locations 
                           for nbr in nbr_func(row,col)):
                        drow, dcol = self.directions[rule_id]
                        # add this elf's id to the proposal for that (new_row, new_col) as destination
                        self.elf_proposals[(row+drow, col+dcol)].append((self.elf_locations[(row,col)],row,col))
                        break # out of rule_id for loop
                else: # if none of the conditions were met, stay put
                    self.elf_proposals[(row,col)].append((self.elf_locations[(row,col)],row,col))
            else:
                # all neighbors are empty - stay put
                self.elf_proposals[(row,col)].append((self.elf_locations[(row,col)],row,col))
        
        if num_need_to_move == 0:
            print(f"No elves move in round {self.num_rounds_completed + 1}! Process ends")
            return False
        else:
            # second half of the round - move to the proposed locations
            for row,col in self.elf_proposals:
                if len(self.elf_proposals[(row,col)]) == 1: # only one contender for this position - move there
                    self.new_elf_locations[(row,col)] = self.elf_proposals[(row,col)][0][0]
                else:
                    for idx,r,c in self.elf_proposals[(row,col)]:
                        self.new_elf_locations[(r,c)] = idx

            assert self.num_elves == len(self.new_elf_locations)
            # swap dictionaries and clear the 2nd one
            self.elf_locations, self.new_elf_locations = self.new_elf_locations, self.elf_locations
            self.new_elf_locations.clear()

        self.num_rounds_completed += 1
        return True

        

## Part 1



In [5]:
ground = Ground(get_input_data(test=1))
for round_num in range(10):
    ground.process_round()
print(ground.count_empty_tiles())

110


In [6]:
ground = Ground(get_input_data(test=0))
for round_num in range(10):
    ground.process_round()
print(ground.count_empty_tiles())

3871


In [7]:
puzzle.answer_a = 3871
puzzle.answer_a

'3871'

## Part 2



In [8]:
ground = Ground(get_input_data(test=1))
while ground.process_round():
    pass


No elves move in round 20! Process ends


In [9]:
ground = Ground(get_input_data(test=0))
while ground.process_round():
    pass


No elves move in round 925! Process ends


In [10]:
puzzle.answer_b = 925
puzzle.answer_b

'925'